# Physican Compare Analysis (2017)
Allison Black
IE Master's in Buisness Analytics and Big Data


Physician Compare analysis is going to be performed as follows:

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Get one or multiple **random samples** from the data set to better understand what the data is all about
  3. Identify **data entities**, **metrics** and **dimensions**
  4. **Columns/fields categorization**
4. Columns groups **basic profiling** to better understand our data set:
  1. **Performance related** columns basic profiling
  2. **Location related** columns basic profiling
5. **Answer some business questions** to improve patient helath and service
  1. **Ratio of hospital measure performance rate by grade**
  2. **Measure grade statistics** by type of measure (e-prescribing, secure messaging, pain assessment and follow-up)
  3. **Top 100 best and worst hospitals by average performance measure**



## 1. PySpark environment setup

In [219]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# Refer to https://spark.apache.org/docs/latest/api/python/index.html for PySpark documentation

## 2. Data source and Spark data abstraction (DataFrame) setup

In [220]:
healthDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("Physician_Compare_2017_Group_Public_Reporting_-_Measures.csv")
healthDF.head()

Row(Hospital='ALASKA CENTER FOR EAR NOSE AND THROAT INC', Group PAC ID=7012901952, State='AK', ACO PC ID 1=None, Measure Code='MIPS_GRP_093_overall', Measure Title='Acute Otitis Externa (AOE): Systemic Antimicrobial Therapy - Avoidance of Inappropriate Use', Measure Performance Rate=100, Denominator Count=24, Star Value=None, Five Star Benchmark=None, Reported on PC Live Site='N')

## 3. Data set metadata analysis
### A. Display schema and size of the DataFrame

In [221]:
from IPython.display import display, Markdown

healthDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % healthDF.count()))

root
 |-- Hospital: string (nullable = true)
 |-- Group PAC ID: long (nullable = true)
 |-- State: string (nullable = true)
 |-- ACO PC ID 1: string (nullable = true)
 |-- Measure Code: string (nullable = true)
 |-- Measure Title: string (nullable = true)
 |-- Measure Performance Rate: integer (nullable = true)
 |-- Denominator Count: integer (nullable = true)
 |-- Star Value: integer (nullable = true)
 |-- Five Star Benchmark: integer (nullable = true)
 |-- Reported on PC Live Site: string (nullable = true)



This DataFrame has **73321 rows**.

### B. Get two random samples from the data set

In [222]:
healthDF.cache() # optimization to make the processing faster
healthDF.sample(False, 0.8).take(2)

[Row(Hospital='MAT-SU VALLEY MEDICAL CENTER LLC', Group PAC ID=5698686475, State='AK', ACO PC ID 1=None, Measure Code='MIPS_GRP_093_overall', Measure Title='Acute Otitis Externa (AOE): Systemic Antimicrobial Therapy - Avoidance of Inappropriate Use', Measure Performance Rate=53, Denominator Count=97, Star Value=None, Five Star Benchmark=None, Reported on PC Live Site='N'),
 Row(Hospital='TANANA VALLEY CLINIC LLC', Group PAC ID=5698056901, State='AK', ACO PC ID 1=None, Measure Code='MIPS_GRP_093_overall', Measure Title='Acute Otitis Externa (AOE): Systemic Antimicrobial Therapy - Avoidance of Inappropriate Use', Measure Performance Rate=75, Denominator Count=118, Star Value=None, Five Star Benchmark=None, Reported on PC Live Site='N')]

### C. Data entities, metrics and dimensions

I've identified the following elements:

* **Entities:** Hospital (main one which is measured - facts), State (dimension), Measure Title (dimension)
* **Metrics:** Measure Performance Rate, Star Value, Five Star Benchmark
* **Dimensions:** Location, measure code, collection type 

### D. Column categorization

The following could be a potential column categorization:

* **Performance related columns:** *Measure title*, *Measure Performance Rating*, *Star Value*, *Five Star Benchmark*
* **Location related columns:** *Hospital*, *State*, *Group PAC ID*


## 4. Columns groups basic profiling to better understand the data set
### A. Performance related columns basic profiling

In [223]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit


print ("Summary of columns Measure Performance Rate, Star Value, Five Star Benchmark:")
healthDF.select("Measure Performance Rate", "Star Value", "Five Star Benchmark" ).summary().show()

print("Checking for nulls on columns Measure Performance Rate, Star Value, Five Star Benchmark:")
healthDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Measure Performance Rate","Star Value","Five Star Benchmark"]]).show()

print("Checking amount of distinct values in columns Measure Performance Rate, Star Value, Five Star Benchmark:")
healthDF.select([countDistinct(c).alias(c) for c in ["Measure Performance Rate","Star Value","Five Star Benchmark"]]).show()


Summary of columns Measure Performance Rate, Star Value, Five Star Benchmark:
+-------+------------------------+------------------+-------------------+
|summary|Measure Performance Rate|        Star Value|Five Star Benchmark|
+-------+------------------------+------------------+-------------------+
|  count|                   52828|              3196|               3196|
|   mean|      62.528621185734835|3.5259699624530665|  98.15425531914893|
| stddev|      38.124502934221155|1.2387641492890453|  4.051891581973628|
|    min|                       0|                 1|                 84|
|    25%|                      22|                 3|                100|
|    50%|                      80|                 4|                100|
|    75%|                      98|                 4|                100|
|    max|                     100|                 5|                100|
+-------+------------------------+------------------+-------------------+

Checking for nulls on columns Mea

### B. Location related columns basic profiling 

In [224]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

print ("Summary of columns Hospital, Group PAC ID,  State, Reported on PC Live Site:")
healthDF.select("Hospital", "Group PAC ID", "State", "Reported on PC Live Site").summary().show()

print("Checking for nulls on columns Hospital, Group PAC ID,  State, Reported on PC Live Site:")
healthDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Hospital", "Group PAC ID", "State", "Reported on PC Live Site"]]).show()

print("Checking amount of distinct values in columns Hospital, Group PAC ID, State, Reported on PC Live Site:")
healthDF.select([countDistinct(c).alias(c) for c in ["Hospital", "Group PAC ID", "State", "Reported on PC Live Site"]]).show()

print ("Most and least frequent occurrences for Hospital, Group PAC ID, State, Reported on PC Live Site:")
HospitalDF = healthDF.groupBy("Hospital").agg(count(lit(1)).alias("Total"))
Group_PAC_DF = healthDF.groupBy("Group PAC ID").agg(count(lit(1)).alias("Total"))
StateDF = healthDF.groupBy("State").agg(count(lit(1)).alias("Total"))
Reported_Live_DF = healthDF.groupBy("Reported on PC Live Site").agg(count(lit(1)).alias("Total"))

leastFreqHospital     = HospitalDF.orderBy(col("Total").asc()).first()
mostFreqHospital      = HospitalDF.orderBy(col("Total").desc()).first()
leastFreqGroupPAC     = Group_PAC_DF.orderBy(col("Total").asc()).first()
mostFreqGroupPAC      = Group_PAC_DF.orderBy(col("Total").desc()).first()
leastFreqState        = StateDF.orderBy(col("Total").asc()).first()
mostFreqState         = StateDF.orderBy(col("Total").desc()).first()
leastFreqReportedLive = Reported_Live_DF.orderBy(col("Total").asc()).first()
mostFreqReportedLive  = Reported_Live_DF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqHospital", "mostFreqHospital", "leastFreqGroupPAC", "mostFreqGroupPAC", \
       "%s (%s occurrences)" % (leastFreqHospital["Hospital"], leastFreqHospital["Total"]), \
       "%s (%s occurrences)" % (mostFreqHospital["Hospital"], mostFreqHospital["Total"]), \
       "%s (%d occurrences)" % (leastFreqGroupPAC["Group PAC ID"], leastFreqGroupPAC["Group PAC ID"]), \
       "%s (%d occurrences)" % (mostFreqGroupPAC ["Group PAC ID"], mostFreqGroupPAC ["Group PAC ID"]))))
display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqState", "mostFreqState", "leastFreqReportedLive", "mostFreqDest", \
       "%s (%d occurrences)" % (leastFreqState ["State"], leastFreqState ["Total"]), \
       "%s (%d occurrences)" % (mostFreqState ["State"], mostFreqState ["Total"]), \
       "%s (%d occurrences)" % (leastFreqReportedLive["Reported on PC Live Site"], leastFreqReportedLive["Total"]), \
       "%s (%s occurrences)" % (mostFreqReportedLive["Reported on PC Live Site"], mostFreqReportedLive["Reported on PC Live Site"]))))



Summary of columns Hospital, Group PAC ID,  State, Reported on PC Live Site:
+-------+--------------------+--------------------+-----+------------------------+
|summary|            Hospital|        Group PAC ID|State|Reported on PC Live Site|
+-------+--------------------+--------------------+-----+------------------------+
|  count|               73321|               73321|73321|                   73321|
|   mean|                null| 4.983921843809727E9| null|                    null|
| stddev|                null|2.8811260416445284E9| null|                    null|
|    min|1450 ASSOCIATES, LLC|            42101099|   AK|                       N|
|    25%|                null|          2466545041| null|                    null|
|    50%|                null|          4981731312| null|                    null|
|    75%|                null|          7416960265| null|                    null|
|    max| ZZZ ANESTHESIA, INC|          9931491107|   WY|                       Y|
+-------+-


| leastFreqHospital | mostFreqHospital | leastFreqGroupPAC | mostFreqGroupPAC |
|----|----|----|----|
| ST JOSEPH HOSPITALIST MEDICAL GROUP INC (1 occurrences) | REGENTS OF THE UNIVERSITY OF CALIFORNIA (160 occurrences) | 3476443524 (3476443524 occurrences) | 9436062296 (9436062296 occurrences) |



| leastFreqState | mostFreqState | leastFreqReportedLive | mostFreqDest |
|----|----|----|----|
| GU (12 occurrences) | FL (5665 occurrences) | Y (3552 occurrences) | N (N occurrences) |


In [225]:
# Drop nulls
healthDF = healthDF.dropna(subset=["Measure Performance Rate"])

print("Checking for nulls on columns Measure Performance Rate:")
healthDF.select([count(when(col(c).isNull(),\
        c)).alias(c) for c in ["Measure Performance Rate"]]).show()

Checking for nulls on columns Measure Performance Rate:
+------------------------+
|Measure Performance Rate|
+------------------------+
|                       0|
+------------------------+



## 5. Answer some business questions to improve patient service 

### A. Ratio of hospital measure performance rating by grade 

In [226]:
from pyspark.sql.functions import count, round

# Grade is going to be categorized as follows:
#
#   "excellent"      =(90,100] 
#   "good"           =(80,89] 
#   "average"        =(70,79] 
#   "poor"           =(0,69] 

# 1. Enrich the DF with grades based on our categorization
totalHealth = healthDF.count()

gradeDF = healthDF\
   .where(col("Measure Performance Rate")!=101)\
   .withColumn("Grade", when(col("Measure Performance Rate")>=90,"1. Excellent")\
                 .when((col("Measure Performance Rate")>79) & (col("Measure Performance Rate")<=89),"2. Good")\
                 .when((col("Measure Performance Rate")>69) & (col("Measure Performance Rate")<=79),"3. Average")\
                 .when((col("Measure Performance Rate")>49) & (col("Measure Performance Rate")<=69),"4. Poor")\
                .otherwise("5. Unacceptable"))
gradeDF.cache() # optimization to make the processing faster

# 2. Ready to answer to this business question
gradeDF.where(col("Measure Performance Rate")!=101)\
                     .select("Grade", "Measure Performance Rate")\
                     .groupBy("Grade")\
                     .agg(count("Grade").alias("Instances"), \
                          (count("Grade")/totalHealth*100).alias("Ratio"))\
                    .orderBy("Grade")\
                     .select("Grade","Instances",round("Ratio",2).alias("RoundedRatio")).show()



+---------------+---------+------------+
|          Grade|Instances|RoundedRatio|
+---------------+---------+------------+
|   1. Excellent|    21803|       41.27|
|        2. Good|     4679|        8.86|
|     3. Average|     3107|        5.88|
|        4. Poor|     4311|        8.16|
|5. Unacceptable|    18928|       35.83|
+---------------+---------+------------+



### B. Measure statistics by type of measure (e-prescribing, secure messaging, pain assessment and follow-up) 

In [227]:
from pyspark.sql.functions import max, min, avg, stddev
from pyspark.sql.types import IntegerType

# Statistics on e-prescribing, secure messaging, and pain assessment & follow up

measuresDF = healthDF.select("Measure Performance Rate")

display(Markdown("**E-Prescribing Statistics**:"))
e_rxDF = measuresDF.where(col("Measure Title")==("e-Prescribing"))
e_rxDF.summary().show()

display(Markdown("**Secure Messaging Statistics**:"))
secureDF = measuresDF.where(col("Measure Title")==("Secure Messaging"))
secureDF.summary().show()

display(Markdown("**Pain Assessment and Follow-Up Statistics**:"))
painDF = measuresDF.where(col("Measure Title")==("Pain Assessment and Follow-Up"))
painDF.summary().show()



**E-Prescribing Statistics**:

+-------+------------------------+
|summary|Measure Performance Rate|
+-------+------------------------+
|  count|                    4873|
|   mean|       84.49538272111636|
| stddev|      18.271657786469472|
|    min|                       0|
|    25%|                      79|
|    50%|                      91|
|    75%|                      96|
|    max|                     100|
+-------+------------------------+



**Secure Messaging Statistics**:

+-------+------------------------+
|summary|Measure Performance Rate|
+-------+------------------------+
|  count|                    4383|
|   mean|       18.11088295687885|
| stddev|      21.721727249606282|
|    min|                       0|
|    25%|                       1|
|    50%|                      10|
|    75%|                      28|
|    max|                     100|
+-------+------------------------+



**Pain Assessment and Follow-Up Statistics**:

+-------+------------------------+
|summary|Measure Performance Rate|
+-------+------------------------+
|  count|                     425|
|   mean|                   69.04|
| stddev|      37.373002565417686|
|    min|                       0|
|    25%|                      38|
|    50%|                      90|
|    75%|                     100|
|    max|                     100|
+-------+------------------------+



### C. Top 100 best and worst hospitals 

In [228]:
# Find and order the average Measure Performance Rate of top 100 hospitals (with state)

display(Markdown("**Top 100 Hospitals by average Measure Performance Rate**:"))

hospitalState = healthDF.groupBy("Hospital", "State").mean("Measure Performance Rate")
Top100DF = hospitalState.orderBy(avg("Measure Performance Rate"), ascending = False).limit(100)
Top100DF.show(100)

**Top 100 Hospitals by average Measure Performance Rate**:

+--------------------+-----+-----------------------------+
|            Hospital|State|avg(Measure Performance Rate)|
+--------------------+-----+-----------------------------+
|SUMMIT PATHOLOGY ...|   OH|                        100.0|
|AMSOL ANESTHETIST...|   IL|                        100.0|
|PATHOLOGY LAB ASS...|   AL|                        100.0|
|MID MICHIGAN INTE...|   MI|                        100.0|
|PATHOLOGY ASSOCIA...|   FL|                        100.0|
|RADIOLOGY REGIONA...|   FL|                        100.0|
|WEST RIVER ANESTH...|   SD|                        100.0|
|JELLICO COMMUNITY...|   TN|                        100.0|
|SOLANO ANESTHESIA...|   CA|                        100.0|
|PIKEVILLE MEDICAL...|   KY|                        100.0|
|SOUTHERN WESTCHES...|   NY|                        100.0|
|MARIA NINA CONCEP...|   FL|                        100.0|
|FLOWERS PATHOLOGY...|   AL|                        100.0|
|ASSOCIATED PATHOL...|   CA|                        100.

In [229]:
# Find and order the average Measure Performance Rate of bottom 100 hospitals (with state)

display(Markdown("**Bottom 100 Hospitals by average Measure Performance Rate**:"))

Bottom100DF = hospitalState.orderBy(avg("Measure Performance Rate"), ascending = True).limit(100)
Bottom100DF.show(100)


**Bottom 100 Hospitals by average Measure Performance Rate**:

+--------------------+-----+-----------------------------+
|            Hospital|State|avg(Measure Performance Rate)|
+--------------------+-----+-----------------------------+
|MAGNOLIA EXPRESS ...|   MS|                          0.0|
|PATRIOT URGENT CA...|   MA|                          0.0|
|    GARY L CURSON PA|   FL|                          0.0|
|PAJARO VALLEY NEU...|   CA|                          0.0|
|ALLERGY and ASTHM...|   OH|                          0.0|
|    VALLEY ENT, P.C.|   PA|                          0.0|
|LELWICA CHIROPRACTIC|   MN|                          0.0|
|SYRACUSE ENT SURG...|   NY|                          0.0|
|ORTHOPEDIC INSTIT...|   CA|                          0.0|
|WESTERN INFECTIOU...|   CO|                          0.0|
|PACIFICA EMERGENC...|   CA|                          0.0|
|KENT E DOBBINS OD...|   KS|                          0.0|
|VALLEY PULMONARY ...|   TX|                          0.0|
|ASCENSION OUR LAD...|   WI|                          0.

In [230]:
# Bad states: how many times states show up in the bottom 100

display (Markdown("**Number of times states show up in the bottom 100**:"))

Bottom100DF.groupby("State").count().orderBy("count", ascending=False).show()


**Number of times states show up in the bottom 100**:

+-----+-----+
|State|count|
+-----+-----+
|   FL|   11|
|   CA|    8|
|   NY|    6|
|   GA|    6|
|   IL|    6|
|   TX|    5|
|   CO|    5|
|   WA|    4|
|   PA|    4|
|   AZ|    4|
|   MA|    4|
|   WI|    4|
|   MO|    3|
|   MI|    3|
|   MS|    3|
|   OH|    2|
|   IN|    2|
|   NC|    2|
|   SC|    2|
|   MD|    2|
+-----+-----+
only showing top 20 rows

